[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1blkWFKMYqinrtTSKWO_Uvv7v_TvtX6zs?usp=sharing)

# Lesson 8: Going Deeper into NLP

[Previously](https://github.com/drscotthawley/DLAIE/blob/main/Lessons/7_NLP_via_HuggingFace_Transformers.ipynb) we saw how convenient it was to use the `pipeline` method of the HuggingFace.co `transformers` library to perform a variety of Natural Language Processing (NLP) tasks. But there's a lot going on under the hood that was hidden from us.  If we want to learn how these models work, we're going to have to peel back several layers, on multiple levels.  

What we did in the previous NLP lesson was a bit like watching a big rocket take off from a distance. There are many systems in the rocket that are all working together to effect the launch.  To understand how the big rocket operates, it will help if we go back to study smaller, simpler rockets so that we understand the principles of rocketry. 

In this lesson we'll learn the parts of an NLP model and see how they go together. 

## 1. Tokenization

Whatever NLP task we're interested in performing, there will be a large amount of text (sometimes called a "corpus") that we will use for training the model on. That text needs to be split up somehow into bite-sized parts to operate upon. This process is known as *tokenization*. We could try treating individual characters as tokens, or [regard entire sentences as our tokens](https://claritynlp.readthedocs.io/en/latest/developer_guide/algorithms/sentence_tokenization.html), but a common mid-point is to use *words* as tokens.  

> *For a great example of a character-based neural network, see [Andrej Karpaty's Char-RNN](https://github.com/karpathy/char-rnn). \[OPTIONAL, not required\]

The simplest -- and typically *the default* -- scheme for word-level tokenization is just to split the text at every space and at every punctuation mark. Let's try an example


So for instance, the follwing sample text:
```
I'm going to the store, because I need some milk.
```
might become
```
["I", "'", "m", "going", "to", "the",  "store", ",", "because", "I", "need", "some", "milk", "."]
```
Tokenization is something that many computational linguists have spent a great deal of time on, and there are [a variety of tokenizers](https://towardsdatascience.com/overview-of-nlp-tokenization-algorithms-c41a7d5ec4f9?gi=73a2ec14356e) available. Generally it's generally in our best interest to just call a library such as[Natural Language Toolkit (NLTK)](https://www.nltk.org/) to do the tokenizing for us instead of trying to do it from scratch. Both FastAI and HuggingFace allow us to choose between a variety of tokenizers.  (FastAI's default tokenizer is currently from the [spaCy NLP library](https://spacy.io/).)

Let's try an actual example using the NLTK word tokenizer:

In [ ]:
import nltk
nltk.download('punkt')    # this is a resource needed by NLTK
sentence = "I'm going to the store, because I need some milk."
tokens = nltk.word_tokenize(sentence)
print("tokens = ",tokens)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
tokens =  ['I', "'m", 'going', 'to', 'the', 'store', ',', 'because', 'I', 'need', 'some', 'milk', '.']


Interesting that the apostrophe from "I'm" went with the "m" (as in "'m") instead of being its own thing. Presumably this is so we can then expand it into "am".  What about the "n" in "don't"? 

In [ ]:
sentence2 = "I don't know what's going to happen in this case, but it should be interesting!"
tokens = nltk.word_tokenize(sentence2)
print(tokens)

['I', 'do', "n't", 'know', 'what', "'s", 'going', 'to', 'happen', 'in', 'this', 'case', ',', 'but', 'it', 'should', 'be', 'interesting', '!']


In this case the "n" from "don't" went with the "'t". Again, this best facilitates filling in the missing "o".  Let's try some spirited Tennessee-style language:

In [ ]:
sentence3 = "I'm fixin' to spend $1499.95 on a new four wheeler and you ain't gonna stop me, ma!"
print(nltk.word_tokenize(sentence3))

['I', "'m", 'fixin', "'", 'to', 'spend', '$', '1499.95', 'on', 'a', 'new', 'four', 'wheeler', 'and', 'you', 'ai', "n't", 'gon', 'na', 'stop', 'me', ',', 'ma', '!']


Wow, it knows "you'uns"!  And it splits "gonna", presumably in preparation for a mapping to "going", "to".

Do we need the commas and exclamation points though?  Maybe, maybe not.  It depends on our use case.  Sometimes other punctuation is relevant, such as hashtags and @-symbols for social media.  NLTK has a special tokenizer for Twitter:

In [ ]:
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()

tweet = "OMG I love @SuperFamousPerson's new look! #fridays #nofilter"

# Let's compare the two tokenizers:
print("Regular word tokenizer:", nltk.word_tokenize(tweet))
print("Tweet tokenizer:       ",tt.tokenize(tweet))

Regular word tokenizer: ['OMG', 'I', 'love', '@', 'SuperFamousPerson', "'s", 'new', 'look', '!', '#', 'fridays', '#', 'nofilter']
Tweet tokenizer:        ['OMG', 'I', 'love', '@SuperFamousPerson', "'", 's', 'new', 'look', '!', '#fridays', '#nofilter']


...So the specialty `TweetTokenizer` kept certain kinds of punctuation with their associated words, rather then splitting at all forms of punctuation like the regular word tokenizer did.



Beyond the question of which punctuation to keep, we must also recognize that words come in a variety of forms.  And some words may be "filler" that we may not need for the task at hand (e.g., articles like "a", "an", and "the" are often discarded).  So we may wish to regard related words such as "jump", "jumping", "jumps",... as variations on the *stem* of "jump".  We may hang on to the endings such as "-ing" for later use, regarding them as additional tokens. The process of *stemming* or "*stemmification*" is the breaking up of words into their stems and hanging on to endings (or not).  Also, what about compound words?  Some languages such as German will make very long single words (e.g. Geschwindigkeitsbegrenzung for "speed limit") that in other languages would be considered as separate words. If language translation is our goal, some way of tokenizing that includes such variability would be important.  Also, what about punctuation? To keep things simple, we could just delete all forms of punctuation -- or expand contractions like "I'll" to "I will", and so forth -- and yet if we want a highly accurate model we may find that holding on to some forms of punctuation will important.



####  Special Token Codes
Often language models will make use of special tokens such as `UNK` (a token to substitute for unknown words) or `PAD` (for extra padding words), or `EOS` (end of sentence), depending on the task at hand. Sometimes these will have extra characters like `<UNK>` or `[UNK]`. There may or may not be `<START>` and `<END>` tokens for the beginning and end of the text.  The exact list of special tokens depends on the tokenizer and the model, but those few are pretty universal. So when you see those, in what follows, you'll be prepared.  
 





## Numericalization & Word Vectors
Once we have the tokens, we still need to convert these into numbers somehow so we can operate on them mathematically. Depending on the application, different numericalization schemes are available. 

One *very simple* way to do this if we were, say, doing *Sentiment Classification* in tweets, movie reviews, or other kinds of "posts",  would be to count the frequency of all the words that appear in positive posts, and do the same for all the negative posts.  Expressing these frequencies as fractions of the total number of words, we could then assign to each word its pair of "positive use" and "negative use" frequency values $(f_p, f_n)$ which lie in the two-dimensional [unit square](https://en.wikipedia.org/wiki/Unit_square) (shown below). These would then form the coordinates for a *word vector* of our word in its *embedding* space (i.e., the unit square in this case).  Then to classify a post, we could just take the sum of the word vectors of all the words in the post and see whether the result is more "positive" than "negative". In other words, we could ask, which region of the following embedding diagram does the mean of the word vectors in the post lie in?

![img of regions of positive and negative](https://i.imgur.com/mLpQHBj.png)



That might suffice as a simple baseline model, and it might work "ok", but there are issues with it. For example, it's possible that different words could get mapped to the exact same point.  If all you care about is how positive or how negative the post (or tweet, or review) is, this may not be a problem,  but if you want to "understand" the text, produce a translation of it, or generate new text, then this method is useless.  Another issue is that words that mean almost the same thing but are used with different frequencies (e.g. "amazing" and "stupendous") would receive very different word vectors, even though we'd want them to have essentially the same effects on the model's output.

> Terminology: our simplistic method of just summing up the word vectors together pays no attention to the *order* of the words, so the above model would be termed a "Bag of Words" type of model.  

In order to help preserve uniqueness as well as to better allow words to express their ranges of meanings, one typically uses many more than two dimensions for word vector embeddings.  It's quite common to see 256 or more (e.g. 300) dimensions for words.  While these are too many dimensions to visualize (which is why I gave the simple example above!) the computer is able to deal with them just fine.  

The way one typically gets these word vectors is to take in the list of all the (unique) words in the corpus and produce a "vocabulary" which indexes the words and generates a one-hot encoding by treating the words as categories.  Then we map these categories into word vectors via a matrix of trainable weights. So, for example, a corpus with 10,000 unique words mapped into 300-dimensional word vectors would involve a weights matrix of 300\*10000 = 3 million weights. 

\[TODO: Add a picture someday! ;-) \] 

Thus *the "embedding" mapping is itself a neural network* which we train as the front-end of our full (larger) neural network.
This means that the more words you allow in your vocabulary (or "vocab"), the bigger that initial embedding operation will be.  Typically, in order to keep this matrix from getting too big, one will truncate the list of words by removing the less frequent or less important words from the vocab and replacing them with special tokens such as `UNK`. 
The form the embedding takes may depend on the task.  

## Language Modeling as a Pretraining Task
One very useful method is to use a *language model* task to produce word embeddings.  A language model tries to predict the next word in a sequence given its preceding words (how many preceding words you use determines the sophistiation of the model). This forms a "self-supervised learning" method in the sense that the target data you train on is the same as the input data, just shifted ahead by one word. 

This approach was used to great effect by Jeremy Howard and Sebastian Ruder in their [ULMFit paper](https://paperswithcode.com/method/ulmfit), in which they used a language model task of predicting the next word in Wikipedia (specifically, the [Wikitext-103](https://paperswithcode.com/dataset/wikitext-103) dataset) in order to condition the model to use for other tasks such as sentiment analysis of IMDB movie reviews.  Their result was that they beat other competing sentiment analysis methods by a longshot!  

The idea is that a model that has to predict the next word in a large text has to develop somewhat of an "understanding" of how language works, and thus will be a more powerful model for text classification than a simpler model that 

> Note: A neat effect of this form of pre-training is that you also end up with a text generation model.

Now, we're not going to train a model on Wikipedia right now.  That would be a waste of time, as we can just download pretrained weights and go from there.  Let's use the fastai set of methods for doing this, and we'll work through their IMDB example problem [as described in Chapter 10 of the `fastbook`](https://github.com/fastai/fastbook/blob/master/10_nlp.ipynb).  To get started we'll need to download the dataset and start using fastai's tokenizer(s).

In [ ]:
!pip install -Uqq fastai fastbook

In [ ]:
# if the next line produces an error, restart the runtime and try again.
import fastbook  
from fastai.text.all import *
from IPython.display import display, HTML

In [ ]:
 path = untar_data(URLs.IMDB)  # download the dataset

In [ ]:
# make a list of all the files in all the folders of the dataset
files = get_text_files(path, folders = ['train', 'test', 'unsup'])

# let's look at the first 75 characters of the first file in the list
txt = files[0].open().read();  txt[:75]

'Kannathil Muthamittal is for sure a great movie. I have to give it to Mani '

As we mentioned above the current default tokenizer in FastAI is from the spaCy NLP package:

In [ ]:
spacy = WordTokenizer()
spacified = spacy([txt])  
print(spacified)

<generator object SpacyTokenizer.__call__.<locals>.<genexpr> at 0x7f9f15d77f50>


So the word tokenizer is a generator. In order to access its output we can use `first()` and `next()`:


In [ ]:
toks = first(spacy([txt]))
print(toks)   # This prints out all the tokens
print(coll_repr(toks, 30))  # fastai's coll_repr method gives the total size and first N (=30) tokens

['Kannathil', 'Muthamittal', 'is', 'for', 'sure', 'a', 'great', 'movie', '.', 'I', 'have', 'to', 'give', 'it', 'to', 'Mani', 'Ratnam', 'for', 'a', 'great', 'directing', 'job', 'and', 'A.R.', 'Rahman', 'for', 'great', 'songs', '.', 'The', 'camera', 'work', 'is', 'just', 'excellent', 'and', 'is', 'similar', 'to', 'Black', 'Hawk', 'Down', 'and', 'Saving', 'Private', 'Ryan', '.', 'I', 'will', 'be', 'shocked', 'if', 'this', 'movie', 'does', 'not', 'win', 'an', 'Oscar', 'for', 'Best', 'Foreign', 'Film', 'or', 'even', 'Best', 'Camera', 'Work', '.']
(#69) ['Kannathil','Muthamittal','is','for','sure','a','great','movie','.','I','have','to','give','it','to','Mani','Ratnam','for','a','great','directing','job','and','A.R.','Rahman','for','great','songs','.','The'...]


In addition to `WordTokenizer`, fastai adds some extra functionality via a `Tokenizer` method, that will turn all words to lower case but precede such interventions with a special code `xxmaj` indicating that the next word should be capitalized.  It also adds `xxbos` to denote the beginning of the sentence. 

In [ ]:
tkn = Tokenizer(spacy)
print(coll_repr(tkn(txt), 31))

(#91) ['xxbos','xxmaj','kannathil','xxmaj','muthamittal','is','for','sure','a','great','movie','.','i','have','to','give','it','to','xxmaj','mani','xxmaj','ratnam','for','a','great','directing','job','and','xxup','a.r','.'...]


> Note: fastai also has a tokenization method that will use sub-words -- i.e., groups of characters -- but we're going to skip that part for now.

To do calculations on the GPU, it's helpful to work with "batches" of data, just like we did for images.  In each batch we need the same demensions, so we will chop the text up into "chunks" of length `seq_len` and then group these into batches.  Rather than totally randomly assigning the order of the batches, we will have the model "read" the text sequentially, where each new element of a batch will simply be shifted ahead one word. 

See this fastai example where they use a batch size of `bs=6` and sequence length of `seq_len=5` to produce one batch from a sample text:

In [ ]:
stream = "In this chapter, we will go back over the example of classifying movie reviews we studied in chapter 1 and dig deeper under the surface. First we will look at the processing steps necessary to convert text into numbers and how to customize it. By doing this, we'll have another example of the PreProcessor used in the data block API.\nThen we will study how we build a language model and train it for a while."
print(stream)
tokens = tkn(stream)
print("\n",len(tokens),"tokens in stream.")

In this chapter, we will go back over the example of classifying movie reviews we studied in chapter 1 and dig deeper under the surface. First we will look at the processing steps necessary to convert text into numbers and how to customize it. By doing this, we'll have another example of the PreProcessor used in the data block API.
Then we will study how we build a language model and train it for a while.

 90 tokens in stream.


Although we could randomly grab "chunks" from all over the file and try to predict the word following each chunk, the fastai folks recommend making the text in each row of each batch follow immediately from the text in the corresponding row the previous batch.  Which means making some fancy slicing code like the following, in which we show three sequential batches.   

In [ ]:

bs,seq_len = 6, 5                          # batch size and sequence length
num_batches = len(tokens)// bs // seq_len  # 30 tokens per batch, 90 tokens = 3 batches. 
print("num_batches = ",num_batches)  
num_rows = len(tokens) // seq_len          # total rows of all batches == 18
print("num_rows = ",num_rows)

for b in range(num_batches):
    stride = seq_len * num_batches 
    d_tokens = np.array([tokens[i*stride + b*seq_len :i*stride + b*seq_len + seq_len] for i in range(bs)]) # i is the row number
    df = pd.DataFrame(d_tokens)
    print(f"\nbatch = {b}:")
    display(HTML(df.to_html(index=False,header=None)))


num_batches =  3
num_rows =  18

batch = 0:


xxbos,xxmaj,in,this,chapter
movie,reviews,we,studied,in
first,we,will,look,at
how,to,customize,it,.
of,the,preprocessor,used,in
will,study,how,we,build



batch = 1:


",",we,will,go,back
chapter,1,and,dig,deeper
the,processing,steps,necessary,to
xxmaj,by,doing,this,","
the,data,block,xxup,api
a,language,model,and,train



batch = 2:


over,the,example,of,classifying
under,the,surface,.,xxmaj
convert,text,into,numbers,and
we,'ll,have,another,example
.,\n,xxmaj,then,we
it,for,a,while,.


See how each row of each batch continues the text from the same row in the preceding batch?  Don't worry, you won't have to reproduce that code, fastai will do it internally.  

When we were training images, we shuffled the order of images between epochs.  In the case of NLP we don't want to shuffle the words or even the rows.  Instead when we take a bunch of movie reviews and concatenate them to form a stream (which then broken into tokens and then batches), what we do is randomize the *order in which the reviews are concatenated* at each epoch.  This allows for word orderings to stay the same but where they appear in the training dataset to still shift around a bit in order to prevent overfitting. 


This is generally handled automatically by fastai, that will define the Tokenizer, set it up, and specify a Numericalize function, and set that up.  Here we show a brief example of that:

In [ ]:
txts = L(o.open().read() for o in files[:2000])  # read texts of the first 2000 files
txts[0]

'Kannathil Muthamittal is for sure a great movie. I have to give it to Mani Ratnam for a great directing job and A.R. Rahman for great songs. The camera work is just excellent and is similar to Black Hawk Down and Saving Private Ryan. I will be shocked if this movie does not win an Oscar for Best Foreign Film or even Best Camera Work.'

In [ ]:
toks200 = txts[:200].map(tkn)   # tokenize the first 200 files, by mapping the "tkn" function to the elements of text.
toks200[0]  # show us the tokens corresponding to the text in the first file 

(#91) ['xxbos','xxmaj','kannathil','xxmaj','muthamittal','is','for','sure','a','great'...]

In [ ]:
num = Numericalize()
num.setup(toks200)   # create a vocab for the stream we've created. 
coll_repr(num.vocab,20)  # show the first 20 words in the vocab, in order of descending frequency

"(#1904) ['xxunk','xxpad','xxbos','xxeos','xxfld','xxrep','xxwrep','xxup','xxmaj','the',',','.','and','a','of','to','is','in','it','i'...]"

Then we can show how these individual tokens are rendered as numbers.  Note that the special codes get mapped to zero:

In [ ]:
nums200 = toks200.map(num);
print(toks200[0])
print(nums200[0])

['xxbos', 'xxmaj', 'kannathil', 'xxmaj', 'muthamittal', 'is', 'for', 'sure', 'a', 'great', 'movie', '.', 'i', 'have', 'to', 'give', 'it', 'to', 'xxmaj', 'mani', 'xxmaj', 'ratnam', 'for', 'a', 'great', 'directing', 'job', 'and', 'xxup', 'a.r', '.', 'xxmaj', 'rahman', 'for', 'great', 'songs', '.', 'xxmaj', 'the', 'camera', 'work', 'is', 'just', 'excellent', 'and', 'is', 'similar', 'to', 'xxmaj', 'black', 'xxmaj', 'hawk', 'xxmaj', 'down', 'and', 'xxmaj', 'saving', 'xxmaj', 'private', 'xxmaj', 'ryan', '.', 'i', 'will', 'be', 'shocked', 'if', 'this', 'movie', 'does', 'not', 'win', 'an', 'xxmaj', 'oscar', 'for', 'xxmaj', 'best', 'xxmaj', 'foreign', 'xxmaj', 'film', 'or', 'even', 'xxmaj', 'best', 'xxmaj', 'camera', 'xxmaj', 'work', '.']
TensorText([   2,    8,    0,    8,    0,   16,   30,  273,   13,   72,   29,   11,   19,   45,   15,  223,   18,   15,    8,    0,    8,    0,   30,   13,   72,  469,  297,   12,    7,    0,   11,    8,
           0,   30,   72,  470,   11,    8,    9,  426, 

^Note how the unknown / low frequency words get mapped to 0, which is the code for `UNK` (or "xxunk" in fastai parlance).

These can then go into a fastai DataLoader which has been setup for language modeling, [`LMDataLoader`](https://docs.fast.ai/text.data.html#LMDataLoader), which is designed to load a batch of text as an input and the *same text shifted ahead by one word* as the target data

In [ ]:
dl = LMDataLoader(nums200)

# test it
x,y = first(dl)
print(x.shape,y.shape)

# we can print out x & y but lets convert them from numbers to text when we view them
print(', '.join(num.vocab[o] for o in x[0][:20]))
print(', '.join(num.vocab[o] for o in y[0][:20]))

torch.Size([64, 72]) torch.Size([64, 72])
xxbos, xxmaj, xxunk, xxmaj, xxunk, is, for, sure, a, great, movie, ., i, have, to, give, it, to, xxmaj, xxunk
xxmaj, xxunk, xxmaj, xxunk, is, for, sure, a, great, movie, ., i, have, to, give, it, to, xxmaj, xxunk, xxmaj


See how each word in y is just the corresponding "next word in x" at the same index?  As a simple exercise, can you do the same?  Write a "shift left" function that just shifts a set of list elements to the left.  Add a "xxpad" on the end:

In [ ]:
## EXERCISE. Fill in your code below as directed

def shift_left(orig:list):   
    ## Your code below. Define a variable called "shifted" that is the original 
    #  list, shifted to the left by one, and filled in with a "xxpad" at the end.
 
    shifted =  
 
    ## end of your code
    return shifted 

Test your code:

In [ ]:
shift_left([1,2,3,4,5])

[2, 3, 4, 5, 'xxpad']

```
Expected ouput:
[2, 3, 4, 5, 'xxpad']
```

In [ ]:
# and another check
assert shift_left([]) == ['xxpad']

---
## Half-Way Point

*(Ok, more like the 1/3 Point)*

---

## More Exercises!

Huggingface and fastai will end up hiding a lot of what's happening from us, so let's try writing a few more simple helper routines of our own so that we get a feel for what's involved.  The following will be graded. 


### Exercise 8.1: `count_freqs`
Given a list, count up the number of times that each element appears in the list.  Return this as a Python dict called `freqs`:

Note that this can be done as a one-liner using `Counter` from the builtin Python `collections` library, or you can write something similar from scratch yourself.


In [ ]:
## GRADED EXERCISE 8.1
from collections import Counter

def count_freqs(tokens:list):
    ## YOUR CODE HERE

    ### END OF YOUR CODE
    return freqs

Here's some code to check yourself:

In [ ]:
test_list = ['a','b','c','a','d','z','z','q','z','b']
freqs = count_freqs(test_list); freqs

Expected output (note that your order may be different because dicts don't preserve order, but the values should be the same):
```
Counter({'a': 2, 'b': 2, 'c': 1, 'd': 1, 'q': 1, 'z': 3})
```
or
```
dict({'a': 2, 'b': 2, 'c': 1, 'd': 1, 'q': 1, 'z': 3})
```


In [ ]:
# another test:
assert freqs['z'] == 3

### Exercise 8.2: `sort_by_freq`
Given a list, sort its elements in **descending** order of frequency. You should call `count_freqs` in this function.  [Here's a hint](https://stackoverflow.com/questions/613183/how-do-i-sort-a-dictionary-by-value)

In [ ]:
## GRADED EXERCISE 8.2
def sort_by_freq(tokens:list):
    #call count_freqs to get the frequencies
    freqs = ...

    # then sort the tokens according to freqz
    sorted_tokens = 
    
    return sorted_tokens

Test code for you:

In [ ]:
assert sort_by_freq(test_list) == ['z', 'a', 'b', 'c', 'd', 'q']


### Exercise 8.3: `set_vocab_codes`
This will be akin to the "setup" method of fastai's Numericalize: Given an input text,...

1. Tokenize it via the defined `tokenize` method. This will give you a list we'll call `tokens`. 
2. Then rank `tokens` in decreasing order of frequency of their occurance in the text.  Call your `sort_by_freq` function for this.
3. Truncate the list of tokens and only keep the top `keep_frac` fraction of it.
4. Add an 'xxunk' token at the beginning of the list of tokens. 
5. Finally produce a Python `dict` called `vocab_codes` that will map tokens to their index on the sorted list. 

Also, make sure that any unknown words applied to `vocab_codes` return as a [default dict value](https://stackoverflow.com/questions/52195897/how-to-create-a-dict-that-can-account-for-unknown-keys) the code for `xxunk`. 

> Note: The fastai/spacy tokenizer is setup as a *generator*, which is not helpful for this exercise. For this reason we'll use NLTK's tokenizer instead. 

In [ ]:
from fastai.text.all import *
from collections import defaultdict
import nltk
nltk.download('punkt')    # this is a resource needed by NLTK

In [ ]:
## GRADED EXERCISE 8.3

def set_vocab_codes(text:string, tokenizer=nltk.word_tokenize, keep_frac=0.5):
    # INSERT YOUR OWN CODE BELOW
    # 1. Tokenize text via the defined `tokenizer` method. This will give you a list we'll call `tokens`.
    tokens = ...

    # 2. Then rank `tokens` in decreasing order of frequency of their occurance in the text.  Call your sort_by_freq()
    tokens = ...

    # 3. Truncate the list of tokens and only keep the top `keep_frac` fraction of it.
    tokens = ...

    # 4. Add an 'xxunk' token at the beginning of the ranked list of tokens. 
    tokens = ...

    # 5. Finally produce a Python `dict` called `vocab_codes` that will map tokens to their index on the sorted list. 
    vocab_codes = ...

    # Also, (You may want to do this before #5) Make sure that any unknown words applied to `vocab_codes` return as a default 


    ## END OF YOUR CODE
    return vocab_codes

In [ ]:
text = 'The quick brown fox jumped over the lazy dog'
codes = set_vocab_codes(text)
codes

Expected output:    Your codes dict may have a different order than this, but the values should be the same:

```
defaultdict(<function __main__.set_vocab_codes.<locals>.<lambda>>,
            {'The': 1, 'brown': 3, 'fox': 4, 'quick': 2, 'xxunk': 0})
```


In [ ]:
# more tests for you:
assert codes['fox'] == 4
assert codes['Kwisatz Haderach'] == 0

In [ ]:
text = 'It was the best of times, it was the worst of times, it was the age of wisdom, it was the age of foolishness, it was the epoch of belief, it was the epoch of incredulity, it was the season of Light, it was the season of Darkness, it was the spring of hope, it was the winter of despair.'
codes = set_vocab_codes(text)
codes

Expected output:  (Again, the dict order may not be the same, but you should see the same values)

```
defaultdict(<function __main__.set_vocab_codes.<locals>.<lambda>>,
            {',': 4,
             'It': 10,
             'age': 7,
             'best': 11,
             'epoch': 8,
             'it': 5,
             'of': 3,
             'season': 9,
             'the': 2,
             'times': 6,
             'was': 1,
             'xxunk': 0})
```

In [ ]:
assert codes['it'] == 5

You'll notice in the above example that "It" and "it" are treated as two separate words. We could send the whole text in as lowercase to get a different result:

In [ ]:
codes = set_vocab_codes(text.lower())
codes

Expected output: (order may not be the same)
```
defaultdict(<function __main__.set_vocab_codes.<locals>.<lambda>>,
            {',': 5,
             'age': 7,
             'best': 10,
             'epoch': 8,
             'it': 1,
             'of': 4,
             'season': 9,
             'the': 3,
             'times': 6,
             'was': 2,
             'worst': 11,
             'xxunk': 0})
```

...And now "it" is the most frequent non-unk word in the list.   We could still do like fastai and insert a 'xxmaj' code before every capitalization, but... let's move on for now.  


## Exercise 8.4: `codes_to_words`
One other useful thing will be a way to convert from the codes *back* to the words themselves.  Let's create a function that will return a dict in which the keys and values have been swapped.  [Here's a hint](https://www.geeksforgeeks.org/python-program-to-swap-keys-and-values-in-dictionary/)

In [ ]:
## GRADED EXERCISE 8.4 

def codes_to_words(codes:dict):
    ## YOUR CODE BELOW
    words = ...
 
    ## END OF YOUR CODE
    return words 

In [ ]:
words = codes_to_words(codes)
assert words[9] == 'season'
words[0]

### How We Get Word Vector Embeddings

It's incredibly simple: The `dict` variables that map words to codes and codes to words serve as what computational scientists "look up tables". Neural networks don't exactly work with look-up tables but they can work with a very close analogue via matrix multiplication: What we do is take the word codes (/indexes) an use these to denote the rows (or columns) of a *one-hot encoding*, which is a matrix with zeros almost everywhere, and a 1 in each column & row.  The simplest version being just a diagonal matrix of 1's which can be created by functions like Numpy's `eye`.

> Note: Literally, the code you'd write is simply `np.eye(len(tokens))`. It's so simple that we're not even going to make a programming assignment for it, but we *will* do a slighly trickier exercise below.

Say for example we only had 3 words in our vocabulary, "James", "loves", and "tacos".  The one-hot encoding for these could be 

```
'James' = [1, 0, 0]
'loves' = [0, 1, 0]
'tacos' = [0, 0, 1]
```

Then a look-up table could be created when we multiply the *matrix* of one-hot encoded words (i.e.  `np.eye(3)` in this case) with whatever number we want. 

The "number we want" will be a set of numbers, in the form of *weights* of a neural network `Linear` layer (with no activation, i.e. linear activation).  This will produce our word embeddings!  To be clear: the weights will be the same as the activations because we are multiplying "1" by the weights and using no activation.  The dimension of the word vectors produced will be determined by how many dimensions we want in this Linear layer of weights -- for example, we mentioned 300 before.

These weights (i.e. the word embeddings themselves) are initialized randomly and *learned* in the context of training the model for whatever task we want.

You might wonder: is using just one Linear layer with no activation sufficient to accurately map out human language to the point fo being able to produce meaningful embeddings?  In practice, this is remarkably effective, and in fact the `nn.Embedding` layers in both PyTorch and Jupyter are literally just one-hot encoders attached to Linear layers (in Keras they're called "Dense" layers).  

How useful will these embeddings be?  Well, that's an interesting question. "Universal" word embedding such as Word2Vec of GloVe are trained on huge datasets in order to be as general as possible, whereas if you were to simply train on a very small dataset, your embeddings might be only useful for the specific task you want.  Generally, it's useful to start with pretrained embeddings in a "frozen" (non-trainable) state as you train the downstream part of your neural network, and then gradually "unfreeze" the network starting fro the later layers and working backward was the model trains.  (The ULMFiT method referenced earlier describes a detailed way of doing this.)

### Exercise 8.5: `token_to_one_hot`
In this example, we're not going to encode *all* the available tokens at once, rather we're just going to produce the one-hot encodings of the particular words one might find in a sequence.  This will simply involve using the `codes` dict you created to get the index of a word and then forming a one-hot version of that word -- i.e. all zeros except for a 1 at the element corresponding to the word's index/code. The length of the one-hot vector will be the total number of possible words.  So for example, if we had 1000 words, the 'xxunk' would be one-hot encoded as a 1 in the first (0th) spot followed by a list of 999 zeros.  

In the following, use `torch.zeros()` to initialize the vector, with a length of `len(codes)`.

***As an additional requirement: Your routine should return the one-hot vector for 'xxunk' for any token not already assigned a code.*** (You may use recursion to achieve this if you like.)

In [ ]:
## GRADED EXERCISE 8.5
import torch 
def token_to_one_hot(token, codes):
    ### Your code below. Produce a 1D numpy array corresponding to the one-hot vector for token


    ### end of your code
    return onehot_vec

In [ ]:
# a little test code for you
from collections import defaultdict 
test_codes = defaultdict(lambda x:0) 
for key, value in {'xxunk':0, 'the':1, 'apples':2, 'are':3, 'tasty':4}.items():
    test_codes[key] = value

print(token_to_one_hot('apples', test_codes))

Expected output:
```
    tensor([0., 0., 1., 0., 0.])
```

In [ ]:
# More tests:
assert torch.equal( token_to_one_hot('tasty',test_codes), torch.Tensor([0,0,0,0,1]) )

# defaultdict will not automatically handle this next one! Your routine will need to catch it
assert torch.equal( token_to_one_hot('smorgasbord',test_codes), torch.Tensor([1,0,0,0,0]) )

Now, if our "batch" is going to have sequences of words, and each word is a one-hot vector, then won't we end up having a 3-dimensional array for our input?  That was fine for images because we were using 2D convolution operations.  How will we structure the one-hot encodings in our word vectors -- as rows or columns?

Let's try a simple example.  Pretend these words are the tokens. We're going to want to process each word and produce its word vector. 

In [ ]:
test_batch = [['here','is','a','sequence'],['and','here','is','another'],['one','more','sequence','here']]

# Uhhh ok actually our set_vocab_codes needs a string, so we'll convert to an array and then flatten it
batch_array = np.array(batch)
print("batch_array = \n",batch_array)
print("batch_array.shape =",batch_array.shape)
test_text = ' '.join( batch_array.flatten().tolist() )
print("test_text = ",test_text)  # ok, now we've got our string to send to set_vocab_codes

test_codes = set_vocab_codes( test_text )
print("test_codes =",test_codes)

Now we'll produce the one-hot encodings for this batch of sequences:

In [ ]:
onehot_batch = torch.zeros( (batch_array.shape[0], batch_array.shape[1], len(test_codes)) )

for i, row in enumerate(batch_array):
    for j, word in enumerate(row):
        onehot_batch[i, j] = token_to_one_hot(word, test_codes)

(print(onehot_batch) # here the onehot encodings of words will appear along rows

(^Coulda had you do that as an exercise, but imagined it might get confusing. ;-) )

That batch of inputs will then be matrix-multiplied by a set of weights in the Linear layer of a PyTorch model we'll define as we go forward. 

The *output* of that PyTorch model will be a single word, namely a one-hot vector for the next word in the sequence (given by our `shift_left`-ed target data. We will use `softmax` activation and a categorical cross-entropy loss function for this since it's effectively the same thing as predicting one of a variety of categories.  These are just the multi-category versions of sigmoid and binary cross-entropy we saw before. 

> Note: Or even better, we can get a bit more numerical precision if we *don't* use softmax and cross-entropy explicitly but rather use the PyTorch forms of these functions that will avoid any funny exponential blowups (as discussed in our [Santa Claus example](https://hedges.belmont.edu/naughty/) for binary classification). The PyTorch `nn.CrossEntropyLoss` actually expects pre-softmax "logit" values so we won't apply the softmax in our model.   

#### At this point, we've covered all the moving parts of the system except for the model itself!  
We can swap in a variety of models, but for definiteness we're going to use what's called "Recurrent Neural Network" (RNN) that will retain some "memory" of previous "states" when it looked at earlier sequences.  (And *this* is why we made the requirement earlier of having one batch feed directly into the next -- it's because of of this stateful memory).  

The particular form of RNN we'll use is called an LSTM (which stands for "Long Short-Term Memory") and it has a few nice properties for not just remembering things but for "forgetting" things too, and even for "deciding" what's worth remembering or forgetting!

So now, having written most of the components ourselves so we have some idea of "what's going on under the hood," we'll switch over to the fastai versions of them so that we can take advantage of all the other fastai "goodies". 

> Attribution: **In what follows, we will follow a combination of [Chapter 10](https://github.com/fastai/fastbook/blob/master/10_nlp.ipynb) and [Chapter 12](https://github.com/fastai/fastbook/blob/master/12_nlp_dive.ipynb) from the fastai "fastbook"**, almost verbatim.  After all, *is* listed as one of the textbooks for the course. ;-)   We will however change up the order, covering the Chapter 12 content before the Chapter 10 content.

What they do is, first load in a dataset and define their data loaders, then they define and train a few different RNN models ordered by increasing sophistication, finally building up to the LSTM model. (This will be the part from fastbook Chapter 12).  Then we will use this to train a language model and adapt that for text classification (this part will be from fastbook Chapter 10).

Let's take a look at the simplest model they consider first. It's designed to take in three words at a time, which appear in the `.forward()` part of the class as `x[:,0]`, `x[:,1]`, and `x[:,2]`:


> **IMPORTANT:** For what follows, make sure you have GPU acceleration enabled.  Go to `Edit > Notebook settings > Hardware acclerator > GPU`.   This may reset your runtime, in which case you'll need to re-do the install * imports from the top of the notebook, then come back down.  If you don't have the GPU enabled, the training below will take a very long time. [link text](https://) 

In [ ]:
from fastai.text.all import *

class LMModel1(Module):
    def __init__(self, vocab_sz, n_hidden):
        self.i_h = nn.Embedding(vocab_sz, n_hidden)  
        self.h_h = nn.Linear(n_hidden, n_hidden)     
        self.h_o = nn.Linear(n_hidden,vocab_sz)
        
    def forward(self, x):  # SHH: I slightly edited this from fastbook to make it more similar to LMModel2 below
        h = 0
        h = h + self.i_h(x[:,0])
        h = F.relu(self.h_h(h))
        h = h + self.i_h(x[:,1])
        h = F.relu(self.h_h(h))
        h = h + self.i_h(x[:,2])
        h = F.relu(self.h_h(h))
        return self.h_o(h)

So we've got a `nn.Embedding` layer, which as we said earlier is just a (very efficient) proxy for one-hot encoding paired with a linear layer.
After that it's just a 3-layer network, but one in which we add the embeddings (`i_h`) of each word to the "hidden state" (`h`) of the network for this 3-word sequence.  This network has no "memory"; we'll add that later. 

So they setup some data and train this model, and look at the accuracy over a few epochs:

----
## Oct 7, 11:20pm, Pausing here.
----
Anything below is just a sketch

In [ ]:
## TODO: add data set up and model1 training

The next model they do is *exactly the same as model 1* but with the `.forward()` method just written using a loop.  Can you do it yourself (without peeking at the fastbook version)?  

In [ ]:
## GRADED EXERCISE 8.6
class LMModel2(Module):
    def __init__(self, vocab_sz, n_hidden):
        self.i_h = nn.Embedding(vocab_sz, n_hidden)  
        self.h_h = nn.Linear(n_hidden, n_hidden)     
        self.h_o = nn.Linear(n_hidden,vocab_sz)
        
    def forward(self, x):
        h = 0
        for i in range(3):
            ### YOUR CODE BELOW: fill in the two lines needed to perform the same operations as LMModel1


            ### END YOUR CODE.
        return self.h_o(h)

In [ ]:
## todo: train LMModel2 and show it's the same as LMModel1. 

NEXT we finally come to a model that has some "memory": the state `h` is going to persist inside the class (as `self.h`) from one `.forward()` call to the next instead of being reset to 0 every time.  They do include a method to reset the state but it has to be called explicitly, otherwise `self.h` remains whatever value it had *last time* the forward method was called.

In [ ]:
class LMModel3(Module):
    def __init__(self, vocab_sz, n_hidden):
        self.i_h = nn.Embedding(vocab_sz, n_hidden)  
        self.h_h = nn.Linear(n_hidden, n_hidden)     
        self.h_o = nn.Linear(n_hidden,vocab_sz)
        self.h = 0                   # this is the "memory" part. self.h persists with the class after .forward() is finished
        
    def forward(self, x):
        for i in range(3):           # what follows is same as LMModel2 but with "h" replaced by "self.h"
            self.h = self.h + self.i_h(x[:,i])
            self.h = F.relu(self.h_h(self.h))
        out = self.h_o(self.h)
        self.h = self.h.detach()  # for next time: we'll keep the value of h but discard its extra gradient info
        return out
    
    def reset(self): self.h = 0     # we'll call this at the beginning of a new set of text.

(some extra text and such....  then...)

In [5]:
# train this model and show it's better than the previous model(s)! Yay.
# note: this is still not at LSTM. 

wow, that was great.  Should we stop there?

discussion of "create more signal".. 

longer sequences:

In [ ]:
## add some data loading code whereby we build batches with longer sequences
sl = 16    # sequence length
### and stuff

...and a new model:

In [ ]:
class LMModel4(Module):
    def __init__(self, vocab_sz, n_hidden):
        self.i_h = nn.Embedding(vocab_sz, n_hidden)  
        self.h_h = nn.Linear(n_hidden, n_hidden)     
        self.h_o = nn.Linear(n_hidden,vocab_sz)
        self.h = 0
        
    def forward(self, x):
        outs = []
        for i in range(sl):   # Note: sl (sequence length) is a global variable defined above
            self.h = self.h + self.i_h(x[:,i])
            self.h = F.relu(self.h_h(self.h))
            outs.append(self.h_o(self.h))
        self.h = self.h.detach()
        return torch.stack(outs, dim=1)
    
    def reset(self): self.h = 0

In [2]:
# and train the model

...oooh and that's even better, but can we still improve it? 

...discuss...

Let's use PyTorch's RNN class to help us make an even deeper model, where we can specify the depth via `n_layers`:

In [ ]:
class LMModel5(Module):
    def __init__(self, vocab_sz, n_hidden, n_layers):
        self.i_h = nn.Embedding(vocab_sz, n_hidden)
        self.rnn = nn.RNN(n_hidden, n_hidden, n_layers, batch_first=True)
        self.h_o = nn.Linear(n_hidden, vocab_sz)
        self.h = torch.zeros(n_layers, bs, n_hidden)
        
    def forward(self, x):
        res,h = self.rnn(self.i_h(x), self.h)
        self.h = h.detach()
        return self.h_o(res)
    
    def reset(self): self.h.zero_()

In [6]:
# ...and train a deep RNN...

Wait, that was actually WORSE this time?  How come? Isn't deeper better?

...talk about gradients & activations exploding or disappearing...

And finally we come to... 

## LSTM  

(picture)

Looks scary. The parts are really just "tunable switches", implemented using tanh & sigmoid.

This is just one LSTM "cell", kind of like one neuron. Here's the code for it:

In [ ]:
class LSTMCell(Module):
    def __init__(self, ni, nh):
        self.ih = nn.Linear(ni,4*nh)
        self.hh = nn.Linear(nh,4*nh)

    def forward(self, input, state):
        h,c = state
        # One big multiplication for all the gates is better than 4 smaller ones
        gates = (self.ih(input) + self.hh(h)).chunk(4, 1)
        ingate,forgetgate,outgate = map(torch.sigmoid, gates[:3])
        cellgate = gates[3].tanh()

        c = (forgetgate*c) + (ingate*cellgate)
        h = outgate * c.tanh()
        return h, (h,c)

We then chain these cells together. Here's a model that does that:

In [ ]:
class LMModel6(Module):
    def __init__(self, vocab_sz, n_hidden, n_layers):
        self.i_h = nn.Embedding(vocab_sz, n_hidden)
        self.rnn = nn.LSTM(n_hidden, n_hidden, n_layers, batch_first=True)
        self.h_o = nn.Linear(n_hidden, vocab_sz)
        self.h = [torch.zeros(n_layers, bs, n_hidden) for _ in range(2)]
        
    def forward(self, x):
        res,h = self.rnn(self.i_h(x), self.h)
        self.h = [h_.detach() for h_ in h]
        return self.h_o(res)
    
    def reset(self): 
        for h in self.h: h.zero_()

In [ ]:
# and train that...

Wonderful! that's the best so far!

but, can we still do better?


regularlization via dropout

which yields our final model of the day:


In [ ]:
class LMModel7(Module):
    def __init__(self, vocab_sz, n_hidden, n_layers, p):
        self.i_h = nn.Embedding(vocab_sz, n_hidden)
        self.rnn = nn.LSTM(n_hidden, n_hidden, n_layers, batch_first=True)
        self.drop = nn.Dropout(p)
        self.h_o = nn.Linear(n_hidden, vocab_sz)
        self.h_o.weight = self.i_h.weight
        self.h = [torch.zeros(n_layers, bs, n_hidden) for _ in range(2)]
        
    def forward(self, x):
        raw,h = self.rnn(self.i_h(x), self.h)
        out = self.drop(raw)
        self.h = [h_.detach() for h_ in h]
        return self.h_o(out),raw,out
    
    def reset(self): 
        for h in self.h: h.zero_()

In [3]:
# and train...

...oh yeah. that's totally awesome!


## NOW WHAT?

Now that we've trained a LM, we'll cut off its head and use its body for a new model aimed at classifying text. (Chapter 10)